Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [2]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))

df.set_index('id', drop = True, inplace = True)

df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


**Test 2**

In [3]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

In [4]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'],
                                     infer_datetime_format = True)

df['year'] = df['date_recorded'].dt.year

df.drop('date_recorded', axis = 1, inplace = True)

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [5]:
target = 'status_group'

X = df.drop(target, axis = 1)
y = df[target]

**Test 3**

In [6]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = .8)

**Test 4**

In [8]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [9]:
baseline_acc = y_train.value_counts(normalize = True)[0]

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5449074074074074


**Task 5**

In [10]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [11]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import plotly.express as px

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
model = make_pipeline(OrdinalEncoder(handle_missing = np.NaN,
                                    return_df = True),
                     SimpleImputer(),

                     DecisionTreeClassifier(min_samples_split = 20,
                                            min_samples_leaf = 10,
                                            max_depth = 5))


X_train_clean = pd.DataFrame(model[:-1].fit_transform(X_train, y_train), columns = X_train.columns)


model[-1:].fit(X_train_clean, y_train);

In [13]:
feat_imp = pd.Series(model[-1].feature_importances_)
px.bar(x = X_train.columns, y = feat_imp )

**Test 6**

In [14]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [15]:
kbest = SelectKBest(k=10)
kbest.fit_transform(X_train_clean, y_train)
X_train_selected = X_train_clean[X_train_clean.columns[kbest.get_support()]]

X_train_selected.head()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning:

Features [17] are constant.

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning:

invalid value encountered in true_divide



,gps_height,longitude,extraction_type,extraction_type_group,extraction_type_class,payment,payment_type,quantity,quantity_group,waterpoint_type_group
0,0.0,33.867372,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1271.0,34.295074,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,70.0,38.721181,1.0,1.0,1.0,2.0,2.0,3.0,3.0,1.0
3,840.0,37.188388,1.0,1.0,1.0,3.0,3.0,2.0,2.0,1.0
4,0.0,36.458430,3.0,3.0,3.0,3.0,3.0,2.0,2.0,1.0


In [16]:
kbest.pvalues_ , kbest.scores_

(array([2.66056913e-029, 1.91619809e-038, 2.80593729e-135, 2.16699581e-015,
        3.22661828e-147, 4.34544548e-033, 3.95921502e-001, 6.64754943e-001,
        3.93772111e-004, 2.77544752e-001, 1.25720722e-016, 5.38361995e-120,
        7.72247968e-052, 9.51756464e-040, 4.21574549e-060, 8.56616724e-003,
        2.85565971e-061,             nan, 6.33441017e-069, 8.32311269e-001,
        1.58233158e-010, 1.22880922e-033, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 4.76400763e-039, 9.61205873e-025, 0.00000000e+000,
        0.00000000e+000, 7.70426535e-067, 4.30571872e-103, 9.60761168e-145,
        9.60761168e-145, 7.10695883e-080, 9.49889196e-043, 1.30211031e-098,
        2.03893019e-048, 0.00000000e+000, 3.28461036e-024]),
 array([6.58876196e+01, 8.70068179e+01, 3.11846110e+02, 3.37894396e+01,
        3.39714396e+02, 7.46331584e+01, 9.26557381e-01, 4.08340321e-01,
        7.84103182e+00, 1.28180767e+00, 3.66406936e+01, 2.76220203e+02,
        1.17982267e+02, 9.00203777e+01,

In [17]:
px.scatter(x = kbest.pvalues_, y = kbest.scores_)

In [18]:
X_val_clean = pd.DataFrame(model[:-1].transform(X_val), columns = X_val.columns)

training_acc = model.score(X_train_clean, y_train)
validation_acc = model.score(X_val_clean, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.38234427609427607
Validation Accuracy: 0.3918350168350168


**Test 7**

In [19]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [21]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')


X_test['date_recorded'] = pd.to_datetime(X_test['date_recorded'],
                                     infer_datetime_format = True)
X_test['year'] = X_test['date_recorded'].dt.year

X_test.drop('date_recorded', axis = 1, inplace = True)

X_test_clean = pd.DataFrame(model[:-1].transform(X_test),
                            columns = X_test.columns,
                            index = X_test.index)

submission = pd.DataFrame(model[-1:].predict(X_test_clean),
                          columns = ['status_group'],
                          index = X_test_clean.index)

submission.head()

,status_group
id,
50785,non functional
51630,functional
17168,non functional
45559,non functional
49871,functional


**Test 8**

In [22]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [23]:
submission.to_csv('submission.csv')

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.